<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto2/Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparando o ambiente

In [7]:
import pandas as pd
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


##Carregando o dataset e processando os dados

In [16]:
df = pd.read_csv('project1_output.csv')

Vamos utilizar um subset do nosso dataset para diminuir o tempo de experimentação

In [17]:

df = df[:1000]
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.dtypes

year                   int64
acousticness         float64
artists_data          object
danceability         float64
duration_ms            int64
energy               float64
explicit               int64
id                    object
instrumentalness     float64
loudness             float64
name                  object
popularity           float64
release_date          object
speechiness          float64
tempo                float64
first_artist          object
artists_by_artist     object
first_genre           object
dtype: object

Tirando colunas desnecessárias, pois só queremos trabalhar com valores numéricos

In [18]:
df = df.select_dtypes(exclude=['object'])

In [19]:
df.columns

Index(['year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'loudness', 'popularity', 'speechiness',
       'tempo'],
      dtype='object')



####Escolher coluna para predição

In [20]:
target_col = df['popularity']

In [21]:
target_col

0      0.04
1      0.02
2      0.04
3      0.00
4      0.01
       ... 
995    0.00
996    0.00
997    0.00
998    0.00
999    0.00
Name: popularity, Length: 1000, dtype: float64

In [22]:
df = df.drop(columns=['popularity'])

##Separando os dados em teste e predição

Usando uma base 60/20/20

In [ ]:
def get_x_data():
    # input 
    train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    
    return train, val, test

In [ ]:
def get_y_data():
    # output
    train_labels, val_labels, test_labels = (
        np.split(
            target_col, 
            [int(.6*len(target_col)), int(.8*len(target_col))])
    )
    
    return train_labels, val_labels, test_labels

##Escolher os 4 algoritmos para predição

Vamos usar:



*   Regressão Linear
*   Multilayer perceptron
*   random forests
*   lightgbm/xgboost




In [2]:
!pip install mlflow --quiet

     |████████████████████████████████| 14.4 MB 58 kB/s 
     |████████████████████████████████| 1.1 MB 27.1 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 170 kB 57.2 MB/s 
     |████████████████████████████████| 146 kB 39.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 636 kB 37.7 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [4]:
!pip install optuna

     |████████████████████████████████| 302 kB 25.3 MB/s 
     |████████████████████████████████| 80 kB 7.5 MB/s 
     |████████████████████████████████| 111 kB 37.5 MB/s 
     |████████████████████████████████| 141 kB 56.5 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=ce5583930f9674fdda11cc41c652383fe2a3172f2f2bdc274bc583f8c3f3bea5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

import optuna

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import lightgbm
from lightgbm import LGBMRegressor